In [ ]:
import gc
import os
import time
# import argparse
import torch
import torch.nn.functional as F
from glob import glob
from torch.utils.data import DataLoader
from tqdm import trange
from collections import OrderedDict

# from data import AblatingDataset

import utils

In [4]:
def get_text_embeddings(text_encoder, tokenized_text):
    # ref: https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py#L377

    device = text_encoder.device
    weight_dtype = text_encoder.dtype

    text_embedding = text_encoder(tokenized_text.to(device))[0].to(weight_dtype)
    return text_embedding

def get_target_noise(scheduler, noise, latents=None, timesteps=None):
    if scheduler.config.prediction_type == "epsilon":
        target = noise
    elif scheduler.config.prediction_type == "v_prediction":
        target = scheduler.get_velocity(latents, noise, timesteps)
    
    return target

In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class AblatingDataset(Dataset):
    def __init__(
        self,
        data_root,
        tokenizer,
        placeholder_token,
        vae,
        # concept_type:str="object",
        size=512,
        interpolation="bicubic",
        center_crop=False,
        device="cuda:0",
        # batch_size:int=0,
        # is_zero_shot=False
    ) -> None:
        super().__init__()

        self.prompt = placeholder_token
        self.tokenizer = tokenizer
        self.image_embeddings = []
        
        # if not is_zero_shot:
        scaling_factor = 0.18215
        for f in glob(f"{data_root}/*"):
            if ".png" in f or ".jpg" in f or ".jpeg" in f:
                image = utils.preprocess(f, center_crop, size, interpolation).to(device)
                # image = Image.open(f).convert("RGB")
                with torch.no_grad():
                    latents = vae.encode(image).latent_dist.sample().detach().cpu() * scaling_factor
                self.image_embeddings.append(latents[0])
        # else:
        #     for _ in range(batch_size):
        #         self.image_embeddings.append(torch.rand((1, 4, 64, 64)))
        self._length = len(self.image_embeddings)
    
    def __len__(self):
        return self._length

    def __getitem__(self, index):
        # text = random.choice(self.templates).format(self.prompt)
        text = "red"
        tokenized = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt",
        ).input_ids[0]

        return tokenized, self.image_embeddings[index]



In [ ]:

def train():
    
    target_prompt = "red"

    device = "cuda"
    dataset_path = "ds/not_red"
    # dataset_path = "ds/cyan"

    save_path = "R_maximize_Cyan"
    batch_size = 1
    num_epochs = 50
    
    
    tokenizer, text_encoder, vae, unet, scheduler = utils.load_models_from_local_optioned_path(
        text_encoder_path="openai/clip-vit-large-patch14",
        unet_path="models/sd-15/unet",
        vae_path="models/sd-15/vae",
        tokenizer_version="openai/clip-vit-large-patch14",
    )

    unet.to(device)
    text_encoder.to(device)
    vae.to(device)
    vae.eval()
    
    # freeze unet parameters
    for param in unet.parameters():
        param.requires_grad = False

    
    text_encoder = utils.freeze_and_unfreeze_text_encoder(text_encoder, method="mlp-final-attn")

    # optimizer setting
    # optimizer = utils.get_optimizer(
    #     text_encoder.parameters(),
    #     config.optimizer_name,
    #     config.lr,
    #     (config.beta1, config.beta2),
    #     config.weight_decay,
    #     config.eps,
    # )
    optimizer = torch.optim.Adam(text_encoder.parameters(), lr = 1e-5)

    train_dataset = AblatingDataset(
        data_root=dataset_path,
        tokenizer=tokenizer,
        size=512,
        # concept_type=config.concept_type,
        placeholder_token=target_prompt,
        center_crop=False,
        vae=vae,
        device=device,
        batch_size=batch_size,
        # is_zero_shot=config.is_zero_shot
    )


    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    del vae
    gc.collect()
    torch.cuda.empty_cache()
    
    history = {"loss": []}

    os.makedirs(save_path, exist_ok=True)
    start = time.perf_counter()



    pbar = trange(0, num_epochs, desc="Epoch")
    for epoch in pbar:

        loss_avg = 0
        cnt = 0
        text_encoder.train()
        print(f"Starting epoch {epoch}")
        for step, (tokenized, image_embedding) in enumerate(train_dataloader):
            print(f"Processing step {step}")
            text_embedding = get_text_embeddings(
                text_encoder=text_encoder, 
                tokenized_text=tokenized
            )
            
            # bs, 4, 64, 64
            # if zero shot, image_embedding is random noise
            latents = image_embedding.to(device)

            noise = torch.randn_like(latents).to(device)
            bsz = latents.shape[0]
            timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
            timesteps = timesteps.long()
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)
            
            model_pred = unet(noisy_latents, timesteps, text_embedding).sample

            target = get_target_noise(scheduler=scheduler, noise=noise, latents=latents, timesteps=timesteps)
            
            print(f"Device of text_embedding: {text_embedding.device}")
            print(f"Device of latents: {latents.device}")
            print(f"Device of noise: {noise.device}")

            # reversed Textual Inversion, loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
            loss = -F.mse_loss(model_pred.float(), target.float(), reduction="mean") 

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_avg += loss.detach().item()
            cnt += step
            
            history["loss"].append(loss.detach().item())
        pbar.set_postfix(OrderedDict(loss=loss_avg / (cnt + 1e-9)))
        text_encoder.eval()
        text_encoder.save_pretrained(f"{save_path}/epoch-{epoch}")
    
    end = time.perf_counter()
    print(f"Time : {end - start}")

    utils.plot_loss(history, save_path)


In [7]:
train()

d:\AI_project\anaconda3\envs\diffusion\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


text_model.encoder.layers.0.mlp.fc1
text_model.encoder.layers.0.mlp.fc2
text_model.encoder.layers.1.mlp.fc1
text_model.encoder.layers.1.mlp.fc2
text_model.encoder.layers.2.mlp.fc1
text_model.encoder.layers.2.mlp.fc2
text_model.encoder.layers.3.mlp.fc1
text_model.encoder.layers.3.mlp.fc2
text_model.encoder.layers.4.mlp.fc1
text_model.encoder.layers.4.mlp.fc2
text_model.encoder.layers.5.mlp.fc1
text_model.encoder.layers.5.mlp.fc2
text_model.encoder.layers.6.mlp.fc1
text_model.encoder.layers.6.mlp.fc2
text_model.encoder.layers.7.mlp.fc1
text_model.encoder.layers.7.mlp.fc2
text_model.encoder.layers.8.mlp.fc1
text_model.encoder.layers.8.mlp.fc2
text_model.encoder.layers.9.mlp.fc1
text_model.encoder.layers.9.mlp.fc2
text_model.encoder.layers.10.mlp.fc1
text_model.encoder.layers.10.mlp.fc2
text_model.encoder.layers.11.self_attn.k_proj
text_model.encoder.layers.11.self_attn.v_proj
text_model.encoder.layers.11.self_attn.q_proj
text_model.encoder.layers.11.self_attn.out_proj
text_model.encoder.l

d:\AI_project\anaconda3\envs\diffusion\lib\site-packages\diffusers\models\attention_processor.py:2216: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  hidden_states = F.scaled_dot_product_attention(
Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Starting epoch 0
Processing step 0
Device of text_embedding: cuda:0
Device of latents: cuda:0
Device of noise: cuda:0


Epoch:   2%|▏         | 1/50 [00:01<01:28,  1.81s/it, loss=-2.12e+6]

...

Starting epoch 48
Processing step 0
Device of text_embedding: cuda:0
Device of latents: cuda:0
Device of noise: cuda:0


Epoch:  98%|█████████▊| 49/50 [00:31<00:00,  1.69it/s, loss=-1.65e+8]

Starting epoch 49
Processing step 0
Device of text_embedding: cuda:0
Device of latents: cuda:0
Device of noise: cuda:0


Epoch: 100%|██████████| 50/50 [00:32<00:00,  1.54it/s, loss=-5.57e+8]

Time : 32.55622520000907
